# Post-process SUMMA daily outputs
- subset to a particular basin
- extract particular variables

In [1]:
%matplotlib inline

import os, sys
import glob
import xarray as xr
import pandas as pd
import numpy as np

import scripts.utility as util

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])


The Python version: 3.10.12


## 1. settings

In [2]:
main_path = './'
ancillary_path = os.path.join(main_path, 'ancillary_data')
output_path = os.path.join(main_path, 'data', 'summa_mizuRoute')
summa_path  = '/glade/u/home/mizukami/proj/pnw-extrems/models/PNW_GCM'

In [3]:
sim_cases = {
               'gmet_adjusted_t_fixed_4subregion_3_run43':  {'renaming':'gmet', }, 
#               'icar_CanESM5_hist':                {'renaming':'icar_CanESM5_hist',}, 
#               'icar_CanESM5_ssp370':              {'renaming':'icar_CanESM5_ssp370',},
               'icar_NorESM2-MM_ssp370':           {'renaming':'icar_NorESM2-MM_ssp370',},
               'icar_NorESM2-MM_ssp585':           {'renaming':'icar_NorESM2-MM_ssp585',},
              'icar_NorESM2-MM_hist':             {'renaming':'icar_NorESM2-MM_hist',},
               'icar_MPI-M.MPI-ESM1-2-LR_hist':    {'renaming':'icar_MPI-ESM1-2-LR_hist',},
               'icar_MPI-M.MPI-ESM1-2-LR_ssp370':  {'renaming':'icar_MPI-ESM1-2-LR_ssp370',},
            }

sites = ['LIB']

# including variables
variables = {'pptrate_mean':          {'scale':60*60*24,      },
             'airtemp_mean':          {'scale':1,             },
             'scalarTotalRunoff_mean':{'scale':1000*60*60*24, },
             'scalarTotalET_mean':    {'scale':-60*60*24,     },
             'scalarSWE':             {'scale':1,             },
             'hruId':                 {'scale':1,             }
            } 

var_attrs = {'pptrate_mean':            {'units':'mm/day', 'long_name':'precipitation rate (mean)'},
                'airtemp_mean':            {'units':'K',      'long_name':'air temperature at the measurement height (mean)'},
                 'scalarTotalRunoff_mean': {'units':'mm/day', 'long_name':'total runoff (mean)'},
                 'scalarTotalET_mean':     {'units':'mm/day', 'long_name':'total ET (mean)'},
                 'scalarSWE':              {'units':'mm',     'long_name':'snow water equivalent'},
                 'hruId':                  {'units':'-',      'long_name':'ID defining the hydrologic response unit'},
             
            } 

var_encoding = {'pptrate_mean':           {'dtype':'float32', '_FillValue': -9999},
                'airtemp_mean':           {'dtype':'float32', '_FillValue': -9999},
                 'scalarTotalRunoff_mean': {'dtype':'float32', '_FillValue': -9999},
                 'scalarTotalET_mean':    {'dtype':'float32', '_FillValue': -9999},
                 'scalarSWE':             {'dtype':'float32', '_FillValue': -9999},
                 'hruId':                 {'dtype':'int64', '_FillValue': -9999},
                 'time':                  {'dtype':'int64', 'units':'seconds since 1990-1-1 0:0:0.0', '_FillValue': None},
            } 

## Read summa outputs

Read summa daily output, Read HUC12 IDs and extract SUMMA outputs for a basin into ds_summa[sim_case][scenario]

In [4]:
hruids={}
for site in sites:
    hruids[site] = util.read_text(os.path.join(ancillary_path, f'hucIds.{site}.txt'))

In [5]:
%%time

def preprocess(ds):
    ds = ds[[*variables]]
    for var, meta in variables.items():
        ds[var] = ds[var]*meta['scale']
    return ds
    
# get GCM sim
for case, meta in sim_cases.items():
    nclist=glob.glob(os.path.join(summa_path, f'cases/run_{case}/archived/wbout_day_????_????.nc'))
    ds_tmp = xr.open_mfdataset(nclist, data_vars='minimal', parallel=True, preprocess=preprocess).compute()
    ds_tmp['time'] = ds_tmp['time'].dt.round("H")

    for site in sites:
        ds_summa = ds_tmp.where(ds_tmp['hruId'].isin(hruids[site]),drop=True)
    
        for var in ds_summa.variables:
            if var in var_attrs.keys():
                ds_summa[var].attrs = var_attrs[var]
        for var in ds_summa.variables:
            if var in var_encoding.keys():
                ds_summa[var].encoding = var_encoding[var]
        
        ds_summa.to_netcdf(os.path.join(output_path, f"SUMMA_{meta['renaming']}_{site}.nc"))

        print(f'{case}-{site}')

gmet_adjusted_t_fixed_4subregion_3_run43-LIB
icar_NorESM2-MM_ssp370-LIB
icar_NorESM2-MM_ssp585-LIB
icar_NorESM2-MM_hist-LIB
icar_MPI-M.MPI-ESM1-2-LR_hist-LIB
icar_MPI-M.MPI-ESM1-2-LR_ssp370-LIB
CPU times: user 55.1 s, sys: 2min 54s, total: 3min 49s
Wall time: 23min 6s
